## Scenario: In-Store Product Availability Notifications

Imagine you own a store that sells a variety of items.

**Problem:** A customer wants to purchase a specific item, but it's currently out of stock.  You expect to receive more stock in the future. How do you effectively notify the customer when the item becomes available without inconveniencing them or other customers?

**Initial Options:**

1.  **Customer Checks Daily:** The customer could visit the store daily to check for the product's availability.  However, this is inefficient and time-consuming if the item remains unavailable for an extended period.

2.  **Mass Email Notifications:** The store owner could send an email to *all* customers each time a new product becomes available. While potentially helpful for some, this could be perceived as spam by customers not interested in the specific product.

**Desired Solution:**

A subscription mechanism is needed where customers can *choose* when and based on what criteria (e.g., specific product availability, sales events) they want to be notified.  This allows for targeted and relevant notifications, improving customer satisfaction and avoiding unnecessary communication.

In [ ]:
from typing import List

class EmailMsgListener:
    def __init__(self, email: str):
        self.email: str = email

    def update(self):
        print(f"Sending email to: {self.email}")

class Store:
    def __init__(self):
        self.notificationService: NotificationService = NotificationService()

    def newItemPromotion(self):
        self.notificationService.notify()

    def getService(self) -> 'NotificationService':

        return self.notificationService

class NotificationService:
    def __init__(self):
        self.customers: List[EmailMsgListener] = []

    def subscribe(self, listener: EmailMsgListener):
        self.customers.append(listener)

    def unsubscribe(self, listener: EmailMsgListener):
        self.customers.remove(listener)

    def notify(self):
        for listener in self.customers:
            listener.update()


## Note: Observer Pattern Benefits

While a simple loop might initially seem sufficient for notifications, the Observer pattern offers significant advantages in terms of flexibility, maintainability, and adherence to SOLID principles, specifically the Open/Closed Principle.

**Here's a breakdown:**

*   **Open/Closed Principle:** The Observer pattern allows you to add new subscriber types (e.g., SMS notifications, webhooks) without modifying the `Store` or `NotificationService` classes. With a simple loop, you'd have to modify the loop's logic every time you want to add a new notification method. The code becomes tightly coupled and harder to maintain.

*   **Decoupling:** The `Store` class (the publisher) doesn't need to know the specifics of *how* each subscriber handles the notification. It simply calls the `update()` method on each subscriber in the list. This reduces dependencies and makes the system more modular. A simple loop would require the `Store` class to have knowledge of each notification mechanism.

*   **Scalability and Maintainability:** As the number of notification methods and subscriber types grows, the Observer pattern makes it easier to manage the complexity. Adding a new notification type only requires creating a new `EventListener` implementation and subscribing it. The core logic of the `Store` class remains unchanged.

*   **Flexibility for Event Types:** The implemented code showcases event types to be added for subscribing to particular events so the listener will only be notified of that particular event to avoid spam to the users.

**In summary:**

Using the Observer pattern provides a more robust, scalable, and maintainable solution for handling notifications compared to a simple loop. It promotes loose coupling, adheres to SOLID principles, and allows for easy extension as your application evolves. It's a better choice when you anticipate the need for multiple notification methods or subscriber types that might change over time.

## Scenario: Store Notifications with Multiple Listener Types

A store wants to notify customers about new item promotions.  Customers can choose to receive notifications via email or through a mobile app push notification.  The system should be designed to easily support additional notification methods in the future.

![Example](./Image/example.png)

In [ ]:
from abc import ABC, abstractmethod
from typing import List

class EventListener(ABC):
    @abstractmethod
    def update(self):
        pass

class EmailMsgListener(EventListener):
    def __init__(self, email: str):
        self.email: str = email

    def update(self):
        print(f"Sending email to: {self.email}")


class MobileAppListener(EventListener):
    def __init__(self, username: str):
        self.username: str = username

    def update(self):
        print(f"Sending push notification to: {self.username}")

class NotificationService:
    def __init__(self):
        self.customers: List[EventListener] = []

    def subscribe(self, listener: EventListener):
        self.customers.append(listener)

    def unsubscribe(self, listener: EventListener):
        self.customers.remove(listener)

    def notify(self):
        for listener in self.customers:
            listener.update()

class Store:
    def __init__(self):
        self.notificationService: NotificationService = NotificationService()

    def newItemPromotion(self):
        self.notificationService.notify()

    def getNotificationService(self) -> NotificationService:
        return self.notificationService


if __name__ == "__main__":
    store = Store()

    email_listener1 = EmailMsgListener("geekific@like.com")
    email_listener2 = EmailMsgListener("geekific@subs.com")
    mobile_listener1 = MobileAppListener("GeekificLNs")

    store.getNotificationService().subscribe(email_listener1)
    store.getNotificationService().subscribe(email_listener2)
    store.getNotificationService().subscribe(mobile_listener1)

    store.newItemPromotion()

## Scenario: Targeted Notifications for New Items and Sales

A store wants to allow customers to subscribe to specific types of notifications: new item arrivals and sales events. Customers should only receive notifications for the events they've subscribed to, avoiding irrelevant messages.

In [ ]:
from abc import ABC, abstractmethod
from enum import Enum
from typing import List, Dict

# Event Types
class Event(Enum):
    NEW_ITEM = "NEW_ITEM"
    SALE = "SALE"

# Subscriber Interface
class EventListener(ABC):
    @abstractmethod
    def update(self, event_type: Event):
        pass

# Concrete Subscriber: Email Listener
class EmailMsgListener(EventListener):
    def __init__(self, email: str):
        self.email: str = email

    def update(self, event_type: Event):
        print(f"Sending email to: {self.email} for event: {event_type}")

# Publisher: Notification Service
class NotificationService:
    def __init__(self):
        self.customers: Dict[Event, List[EventListener]] = {event: [] for event in Event}

    def subscribe(self, event_type: Event, listener: EventListener):
        self.customers[event_type].append(listener)

    def unsubscribe(self, event_type: Event, listener: EventListener):
        self.customers[event_type].remove(listener)

    def notify(self, event_type: Event):
        for listener in self.customers[event_type]:
            listener.update(event_type)

# Publisher: Store
class Store:
    def __init__(self):
        self.notificationService: NotificationService = NotificationService()

    def newItemPromotion(self, item_name: str):
        print(f"Store: New item {item_name} arrived!")
        self.notificationService.notify(Event.NEW_ITEM)

    def sale_announcement(self, discount: str):
        print(f"Store: Sale with {discount} discount!")
        self.notificationService.notify(Event.SALE)


# Client Code (Example Usage)
if __name__ == "__main__":
    store = Store()

    email_listener1 = EmailMsgListener("geekific@like.com")
    email_listener2 = EmailMsgListener("geekific@subs.com")

    store.notificationService.subscribe(Event.NEW_ITEM, email_listener1)
    store.notificationService.subscribe(Event.SALE, email_listener2)

    store.newItemPromotion("Awesome Gadget")
    store.sale_announcement("20%")

## Observer Pattern Structure and Explanation

The Observer pattern defines a one-to-many dependency between objects, so that when one object (the subject) changes state, all its dependents (observers) are notified and updated automatically.

Here's a breakdown of the key components and how they relate to the store/customer example:

*   **Publisher (Subject):** In our example, this is the `NotificationService`. The publisher maintains a list of subscribers and provides an interface for adding and removing subscribers.  It doesn't need to know the specifics of each subscriber.

*   **Subscription Infrastructure:**  The publisher contains methods for `subscribe()` and `unsubscribe()` which manage the list of listeners.

*   **Notification Process:** When an event occurs (e.g., a new item arrives), the publisher iterates through its list of subscribers and calls the `update()` method on each one.

*   **Subscriber Interface (Observer):** In our example, this is the `EventListener` abstract base class.  It defines the `update()` method, which all concrete subscribers must implement. This method is called by the publisher when an event occurs.

*   **Concrete Subscribers (Observers):** These are the classes that implement the `EventListener` interface (e.g., `EmailMsgListener`, `MobileAppListener`). Each concrete subscriber defines how it responds to a notification.  This could involve sending an email, updating a UI, or performing any other relevant action.

*   **Client:** The client is responsible for creating the publisher and subscriber objects and registering the subscribers with the publisher.  For example, the client might create a `Store` object and then create `EmailMsgListener` and `MobileAppListener` objects and register them with the `Store`'s notification service.

**Key Benefits:**

*   **Loose Coupling:** The publisher and subscribers are loosely coupled. The publisher doesn't need to know the concrete classes of the subscribers.
*   **Open/Closed Principle:** You can add new subscriber types without modifying the publisher's code.
*   **Dynamic Updates:** The set of objects that receive notifications can change dynamically at runtime. Subscribers can register and unregister as needed.
*   **Scalability:**  The Observer pattern allows systems to scale by allowing new subscribers to be added easily.